In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
drive='/content/gdrive/My Drive/python/fever/'  #Change the path to the correct directory where the cleaned_data.csv file is present
os.chdir(drive)
print(os.getcwd())

/content/gdrive/My Drive/python/fever


In [ ]:
!pip install simpletransformers==0.49.0

     |████████████████████████████████| 225kB 14.1MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 1.3MB 21.8MB/s 
     |████████████████████████████████| 7.4MB 52.5MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 317kB 55.2MB/s 
     |████████████████████████████████| 1.8MB 61.2MB/s 
     |████████████████████████████████| 2.9MB 45.9MB/s 
     |████████████████████████████████| 1.1MB 44.5MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
     |████████████████████████████████| 133kB 54.7MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 112kB 66.0MB/s 
     |████████████████████████████████| 6.9MB 41.0MB/s 
     |████████████████████████████████| 4.5MB 48.1MB/s 
     |████████████████████████████████| 163kB 48.3MB/s 
     |████████████████████████████████| 133kB 45.2MB/s 
     |████████████████████████████████| 102kB 10.6MB

In [ ]:
import json
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
def pre_process(sentence):
    ##Replace brackets
    brackets = ['-LRB-', '-LSB-', '-RRB-', '-RSB-']
    for bracket in brackets:
      sentence = sentence.replace(bracket, " ")
    return sentence

In [ ]:
def get_training_data():
    samples_to_use = 368892 #Maximum number of examples is 368892
    label_dict = {"SUPPORTS":0, "REFUTES": 1, "NOT ENOUGH INFO": 2}

    X_all = []
    y_all = []
    count = 0
    #train_file = "formatted_data_train.jsonl"
    train_file = "formatted_data_train_3_class.jsonl"
    fp = open(train_file, 'r')
    for line in fp:
      if count >= samples_to_use:
        break
      obj = json.loads(line.strip())
      claim = pre_process(obj['claim'])
      evidence = pre_process(obj['evidence'])
      X = (claim, evidence)
      y = label_dict[obj['label']]
      X_all.append(X)
      y_all.append(y)
      count += 1
    return X_all, y_all

In [ ]:
(X_all, y_all) = get_training_data()

In [ ]:
print(X_all[:5])
print(y_all[:5])

[('Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'Nikolaj Coster-Waldau then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam   2008   , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot . Fox television Fox Broadcasting Company New Amsterdam New Amsterdam (TV series) Virtuality Virtuality (TV series)'), ('Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'The Fox Broadcasting Company   often shortened to Fox and stylized as FOX   is an American English language commercial broadcast television network that is owned by the Fox Entertainment Group subsidiary of 21st Century Fox . English English language Fox Entertainment Group Fox Entertainment Group 21st Century Fox 21st Century Fox American United States English language English language commercial commercial broadcasting broadcast terrestrial television television network television network'), ('Roman

In [ ]:
#Bring into the format expected by simple transformers
#List of lists. Each list contains sentence1, sentence2 and the class label as shown below
#train_data = [
#    ["Aragorn was the heir of Isildur", "Gimli fought with a battle axe",1,],
#    ["Frodo was the heir of Isildur", "Legolas was an expert archer", 0, ],
#]
X_raw_train, X_raw_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=0)
X_train = []
for X, y in zip(X_raw_train, y_train):
  X_train.append([X[0],X[1], y])
print(X_train[:5])
X_eval=[]
for X, y in zip(X_raw_test, y_test):
  X_eval.append([X[0],X[1], y])
print(X_eval[:5])

[['Emily Blunt starred in a dramatic movie.', 'Emily Blunt received more acclaim for portraying Queen Victoria in The Young Victoria   2009   , receiving a nomination for the Golden Globe Award for Best Actress in a Motion Picture -- Drama . Queen Victoria Queen Victoria The Young Victoria The Young Victoria', 0], ['Jean Grey is a fictional person.', 'Joe Biden chaired the Judiciary Committee during the contentious U.S. Supreme Court nominations of Robert Bork and Clarence Thomas . U.S. Supreme Court U.S. Supreme Court Robert Bork Robert Bork Clarence Thomas Clarence Thomas', 2], ['The National Security Agency has a set of official responsibilities.', 'The NSA is responsible for global monitoring , collection , and processing of information and data for foreign intelligence and counterintelligence purposes , a discipline known as signals intelligence   SIGINT   . counterintelligence counterintelligence signals intelligence signals intelligence', 0], ['Stanley Kubrick directed.', 'Dr. S

In [ ]:
print("#Train={} #Eval={}".format(len(X_train), len(X_eval)))

#Train=295113 #Eval=73779


In [ ]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

In [ ]:
train_df = pd.DataFrame(X_train)
train_df.columns = ["text_a", "text_b", "labels"]
eval_df = pd.DataFrame(X_eval)
eval_df.columns = ["text_a", "text_b", "labels"]

In [ ]:
train_df

,text_a,text_b,labels
0,Emily Blunt starred in a dramatic movie.,Emily Blunt received more acclaim for portrayi...,0
1,Jean Grey is a fictional person.,Joe Biden chaired the Judiciary Committee duri...,2
2,The National Security Agency has a set of offi...,"The NSA is responsible for global monitoring ,...",0
3,Stanley Kubrick directed.,Dr. Strangelove or : How I Learned to Stop Wor...,0
4,Justinian I ruled the Byzantine Empire from 52...,Justinian I dʒʌˈstɪniən Flavius Petrus S...,1
...,...,...,...
295108,Michael Caine was in a trilogy.,Caine played Nigel Powers in the 2002 parody A...,0
295109,Rock music was played by Myles Kennedy.,Myles Richard Kennedy born Myles Richard Bas...,0
295110,Dwyane Wade was in a college team.,After a successful college career at Marquette...,0
295111,Bentley markets luxury shoes.,Rajesh Khanna born Jatin Khanna ; 29 Decembe...,2


In [ ]:
eval_df

,text_a,text_b,labels
0,Batman Begins was released in the 90s.,Batman Begins is a 2005 British-American super...,1
1,Audrey Horne was presented.,"Since 2015 , Lawrence has been the highest-pai...",2
2,Shraddha Kapoor is part of a film industry,Shraddha Kapoor established herself as one of ...,0
3,The Hangover Part III includes alcohol.,The group 's 2016 fourth studio album Glory Da...,2
4,"American Idiot includes the saying ""Boulevard ...",Jones has also portrayed real-life figures suc...,2
...,...,...,...
73774,The Super Nintendo Entertainment System is onl...,"The SNES is Nintendo 's second home console , ...",1
73775,John Lennon was political.,Controversial through his political and peace ...,0
73776,Eminem is a music artist.,Encore stylized as ƎNCORE is the fifth stu...,0
73777,Heikegani is a species of decapod crustacean.,Crabs are decapod crustaceans of the infraorde...,0


In [ ]:
!nvidia-smi

Thu Nov 26 18:03:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#roberta base experiment
# Optional model configuration
args = ClassificationArgs()
args.num_train_epochs=3
args.output_dir = "fever_roberta_base"
args.best_model_dir = "best_fever_roberta_base"
args.save_model_every_epoch = True
# Create a ClassificationModel
model = ClassificationModel("roberta", "roberta-base", num_labels=3, args=args)
# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
X_test = [[s1, s2] for s1, s2, _ in X_eval]
print(X_test[:5])
predictions, raw_outputs = model.predict(X_test)
print(classification_report(predictions, y_test, digits=4))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



{'mcc': 0.951824015033351, 'eval_loss': 0.14984594270600618}
[['Batman Begins was released in the 90s.', 'Batman Begins is a 2005 British-American superhero film based on the DC Comics character Batman , co-written and directed by Christopher Nolan and starring Christian Bale , Michael Caine , Liam Neeson , Katie Holmes , Gary Oldman , Cillian Murphy , Tom Wilkinson , Rutger Hauer , Ken Watanabe , and Morgan Freeman . Christopher Nolan Christopher Nolan DC Comics DC Comics Christian Bale Christian Bale Michael Caine Michael Caine Liam Neeson Liam Neeson Katie Holmes Katie Holmes Gary Oldman Gary Oldman Cillian Murphy Cillian Murphy Tom Wilkinson Tom Wilkinson Rutger Hauer Rutger Hauer Ken Watanabe Ken Watanabe Morgan Freeman Morgan Freeman superhero film superhero film Batman Batman Neeson Liam Neeson Murphy Cillian Murphy'], ['Audrey Horne was presented.', 'Since 2015 , Lawrence has been the highest-paid actress in the world , and her films have grossed over $ 5.5 billion worldwide .


              precision    recall  f1-score   support

           0     0.9772    0.9731    0.9751     38455
           1     0.9296    0.9326    0.9311     13879
           2     0.9856    0.9909    0.9882     21445

    accuracy                         0.9707     73779
   macro avg     0.9641    0.9655    0.9648     73779
weighted avg     0.9707    0.9707    0.9707     73779



In [ ]:
# Optional model configuration
args = ClassificationArgs()
args.num_train_epochs=3
args.output_dir = "fever_xlnet_base_cased"
args.best_model_dir = "best_fever_xlnet_base_cased"
args.save_model_every_epoch = True
# Create a ClassificationModel
model = ClassificationModel("xlnet", "xlnet-base-cased", num_labels=3,args=args)
# Train the model
model.train_model(train_df, eval_df=eval_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
X_test = [[s1, s2] for s1, s2, _ in X_eval]
predictions, raw_outputs = model.predict(X_test)
print(classification_report(predictions, y_test, digits=4))

In [ ]:
!nvidia-smi

Tue Nov 24 18:34:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#xlnet
model_args = ClassificationArgs(num_train_epochs=3,  evaluate_during_training=True, no_save=True, save_best_model=True, save_steps=10000, train_batch_size=32, overwrite_output_dir=True)
# Create a ClassificationModel
model = ClassificationModel("xlnet", "xlnet-base-cased", num_labels=3,args=model_args)
# Train the model
model.train_model(train_df, eval_df=eval_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
X_test = [[s1, s2] for s1, s2, _ in X_eval]
predictions, raw_outputs = model.predict(X_test)
print(classification_report(predictions, y_test, digits=4))

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fbc5b4ee358>>Exception ignored in: 
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fbc5b4ee358>>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
    self._shutdown_workers()  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    
w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers

      File "/usr/lib/python3.6/multiprocessing/process.py", li

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fbc4e188c50>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'Exception ignored in: 
<bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fbc4e188c50>>AssertionError
: Traceback (most recent call last):
can only join a child process  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__

    sel


{'mcc': 0.9583493220282701, 'eval_loss': 0.11079978137936392}



              precision    recall  f1-score   support

           0     0.9812    0.9753    0.9782     38528
           1     0.9365    0.9462    0.9413     13783
           2     0.9875    0.9918    0.9897     21468

    accuracy                         0.9746     73779
   macro avg     0.9684    0.9711    0.9697     73779
weighted avg     0.9747    0.9746    0.9747     73779



In [ ]:
#xlnet experiment
# Optional model configuration
args = ClassificationArgs()
args.num_train_epochs=3
args.output_dir = "fever_xlnet_base_cased"
args.best_model_dir = "best_fever_xlnet_base_cased"
args.save_model_every_epoch = True
# Create a ClassificationModel
model = ClassificationModel("xlnet", "xlnet-base-cased", num_labels=3, args=args)
# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
X_test = [[s1, s2] for s1, s2, _ in X_eval]
print(X_test[:5])
predictions, raw_outputs = model.predict(X_test)
print(classification_report(predictions, y_test, digits=4))